In [48]:
import pandas as pd

In [49]:
ners = ["GENUS", "DISEASE"]

In [50]:
df = pd.read_json(f"../NER/{ner}/test.json", lines=True)


In [51]:
df

,tokens,ner_tags,id
0,"[Except, inducing, priming, ,, we, thought, Os...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",0
1,"[All, of, the, strains, we, analyzed, were, to...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1
2,"[Additionally, ,, a, gene, encoding, a, putati...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",2
3,"[In, contrast, ,, D., alkenigignens, IP3-3^T, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",3
4,"[The, Biosample, ,, Bioproject, and, assembly,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",4
...,...,...,...
306,"[Four, L., aggregata, strains, SDL044, ,, RF14...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",306
307,"[Here, we, describe, the, draft, genome, seque...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",307
308,"[L., masc., adj., caledonicus, ,, from, Caledo...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",308
309,"[Bacillus, subtilis, OhrR, is, a, representati...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",309


In [16]:
test_output

,predictions,label_ids,word_ids
0,"[[-2.478960990905761, -4.24293851852417, 6.236...","[-100, 2, 2, -100, -100, -100, 2, -100, 2, -10...","[None, 0, 1, 1, 1, 1, 2, 2, 3, 3, 3, 4, 5, 5, ..."
1,"[[-2.391100168228149, -4.420975208282471, 6.23...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, -100, -100, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 8, 9, ..."
2,"[[-0.13715796172618802, -3.813025712966919, 3....","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1..."
3,"[[-2.476458311080932, -4.278933048248291, 6.23...","[-100, 2, 2, 2, 2, 2, 2, 2, -100, -100, -100, ...","[None, 0, 1, 2, 3, 4, 5, 6, 6, 6, 6, 6, 7, 8, ..."
4,"[[-2.4910025596618652, -4.118976593017578, 6.2...","[-100, 2, 2, 2, -100, 2, 2, -100, -100, 2, 2, ...","[None, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 7, 8, 9, ..."
...,...,...,...
462,"[[-2.411893129348755, -4.388276100158691, 6.23...","[-100, 2, 2, 2, 2, -100, 2, 2, 2, 2, 2, 2, 2, ...","[None, 0, 1, 2, 3, 3, 4, 5, 6, 7, 8, 9, 10, 11..."
463,"[[-2.484966516494751, -4.157807350158691, 6.21...","[-100, 2, 2, 2, 2, 2, 2, 2, -100, -100, -100, ...","[None, 0, 1, 2, 3, 4, 5, 6, 6, 6, 6, 7, 8, 8, ..."
464,"[[-2.450753688812256, -4.306756973266602, 6.23...","[-100, 2, 2, 2, -100, -100, -100, -100, 2, -10...","[None, 0, 1, 2, 2, 2, 2, 2, 3, 3, 4, 5, 5, 5, ..."
465,"[[-2.470732450485229, -4.256558418273926, 6.23...","[-100, 2, -100, 2, 2, 2, -100, 2, 2, -100, -10...","[None, 0, 0, 1, 2, 3, 3, 4, 5, 5, 5, 5, 6, 7, ..."


In [53]:
l = []
for ner in ["STRAIN"]:
	test_sentences = pd.read_json(f"../NER/{ner}/test.json", lines=True)
	test_output = pd.read_json(f"../NER_output/{ner}/test_outputs.json")
	merged = pd.merge(test_sentences, test_output, left_index=True, right_index=True)
	merged.loc[:,"ner"] = ner
	l.append(merged)

df = pd.concat(l)

In [39]:
df = df.reset_index().drop(columns=["index"])

In [45]:
df

,tokens,ner_tags,id,predictions,label_ids,word_ids,ner
0,"[The, phosphate-solubilizing, P., agglomerans,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",0,"[[-2.478960990905761, -4.24293851852417, 6.236...","[-100, 2, 2, -100, -100, -100, 2, -100, 2, -10...","[None, 0, 1, 1, 1, 1, 2, 2, 3, 3, 3, 4, 5, 5, ...",GENUS
1,"[The, constitutive, PME, expression, displayed...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1,"[[-2.391100168228149, -4.420975208282471, 6.23...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, -100, -100, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 8, 9, ...",GENUS
2,"[The, fact, that, only, a, small, fraction, ,,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",2,"[[-0.13715796172618802, -3.813025712966919, 3....","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1...",GENUS
3,"[Conversely, ,, in, quail, blood, ,, B31-5A4, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",3,"[[-2.476458311080932, -4.278933048248291, 6.23...","[-100, 2, 2, 2, 2, 2, 2, 2, -100, -100, -100, ...","[None, 0, 1, 2, 3, 4, 5, 6, 6, 6, 6, 6, 7, 8, ...",GENUS
4,"[•, That, E., coli, HT115, produces, the, neur...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",4,"[[-2.4910025596618652, -4.118976593017578, 6.2...","[-100, 2, 2, 2, -100, 2, 2, -100, -100, 2, 2, ...","[None, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 7, 8, 9, ...",GENUS
...,...,...,...,...,...,...,...
929,"[DNA, extracted, from, Klebsiella, oxytoca, Am...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",462,"[[-3.156634092330932, -3.220991849899292, 6.94...","[-100, 2, 2, 2, 2, 2, -100, 2, 2, 2, 2, 2, 2, ...","[None, 0, 1, 2, 3, 4, 4, 5, 6, 7, 8, 9, 10, 11...",DISEASE
930,"[These, data, support, the, notion, that, grow...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",463,"[[-1.932698726654052, -3.479667425155639, 5.88...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, -100, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 1...",DISEASE
931,"[PCR, targeting, the, partial, groEL, gene, fr...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",464,"[[-3.2553787231445312, -3.288150548934936, 7.1...","[-100, 2, 2, 2, 2, 2, -100, 2, 2, 2, 2, -100, ...","[None, 0, 1, 2, 3, 4, 4, 5, 6, 7, 8, 8, 8, 9, ...",DISEASE
932,"[In, the, previously, studied, Desulfovibrio, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",465,"[[-3.466292381286621, -3.548609733581543, 7.14...","[-100, 2, 2, 2, 2, 2, -100, -100, -100, 2, 2, ...","[None, 0, 1, 2, 3, 4, 4, 4, 4, 5, 6, 6, 6, 6, ...",DISEASE


In [41]:
df.loc[18,"tokens"]

['The',
 'XCC0613',
 '/',
 'XC_3621',
 'ORF',
 'has',
 'a',
 'G+C',
 'content',
 'of',
 '53.5',
 '%',
 ',',
 'which',
 'is',
 'substantially',
 'lower',
 'than',
 'average',
 'for',
 'other',
 'xanthomonad',
 'genes',
 ',',
 'and',
 'is',
 'suggestive',
 'of',
 'its',
 'acquisition',
 'by',
 'HGT',
 '.']

In [46]:
df.loc[18,"ner_tags"]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [47]:
df.loc[18,"word_ids"]

[None,
 0,
 1,
 1,
 1,
 1,
 2,
 3,
 3,
 3,
 3,
 3,
 4,
 5,
 6,
 7,
 7,
 7,
 8,
 9,
 10,
 10,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 21,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 31,
 32,
 None]

In [43]:
df.loc[18,"label_ids"]

[-100,
 2,
 2,
 -100,
 -100,
 -100,
 2,
 2,
 -100,
 -100,
 -100,
 -100,
 2,
 2,
 2,
 2,
 -100,
 -100,
 2,
 2,
 2,
 -100,
 -100,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 -100,
 -100,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 -100,
 2,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
